import configpip install qgrid
jupyter nbextension enable --py --sys-prefix qgrid
import qgrid



In [1]:
import config
import pymongo
import numpy as np
import pandas as pd

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]

analyseCol = analyseDb[config.COLL_ANALYSE]
metaCollection = stagingDb['Kolominformatie']

In [3]:
grp_aggr = [{"$match" : {'project': {'$nin': ['MAGAZIJN', 'DELF-IT', 'Digifotos']}, 'table': { '$not': {'$regex':"^SYS.*"}}}},
            {"$group": { "_id": { 'project': "$project", 'table': "$table", 'teller': '$teller'}}},
            {'$replaceRoot': {'newRoot': {'project': "$_id.project", 'table': "$_id.table", 'teller': "$_id.teller"}}}]

df = pd.DataFrame(list(metaCollection.aggregate(grp_aggr)))
df['teller'] = pd.to_numeric(df['teller'])
df = df.groupby('table').agg({'project':lambda x: list(x), 'teller':lambda x: sum(x)})
df['Count'] = df.apply(lambda x: len(x.project), axis=1)
df['Stage'] = 'Files'
df = df.sort_values('teller', ascending=False).reset_index()
df_files = df.rename(columns={'table': 'Table', 'project': 'Projecten', 'teller': 'Aantal_Records', 'Count': 'Aantal_tabellen'}).sort_index()
df_files.head(5)

,Table,Projecten,Aantal_Records,Aantal_tabellen,Stage
0,AARDEWERK 1,"[DC97, DB34]",4618,2,Files
1,Aardewerk 1 backup,[DB34],4465,1,Files
2,VONDSTENLIJST,"[DC97, DC11, DC112, DC93, DB34]",2121,5,Files
3,Controle vondsten,[DB34],1466,1,Files
4,DIAOPGRAVING,"[DC93, DC97, DB34]",882,3,Files


In [4]:
grp_aggr = [{"$match" : {'project': {'$nin': ['MAGAZIJN', 'DELF-IT', 'Digifotos']}, 'table': { '$not': {'$regex':"^SYS.*"}}}}
           ,{"$group": { "_id": { 'project': "$project", 'table': "$table"}, 'teller': {"$sum": 1}}}
           ,{'$replaceRoot': {'newRoot': {'project': "$_id.project", 'table': "$_id.table", 'teller': "$teller"}}}
           ]

# pd.concat(frames)
df = pd.concat([pd.DataFrame(list(stagingOud.aggregate(grp_aggr))), pd.DataFrame(list(stagingNieuw.aggregate(grp_aggr)))])
df['teller'] = pd.to_numeric(df['teller'])
df = df.groupby('table').agg({'project':lambda x: list(x), 'teller':lambda x: sum(x)})
df['Count'] = df.apply(lambda x: len(x.project), axis=1)
df['Stage'] = 'Staging'
df = df.sort_values('teller', ascending=False).reset_index()
df_staging = df.rename(columns={'table': 'Table', 'project': 'Projecten', 'teller': 'Aantal_Records', 'Count': 'Aantal_tabellen'}).sort_index()
df_staging.head(5)

,Table,Projecten,Aantal_Records,Aantal_tabellen,Stage
0,AARDEWERK 1,"[DB34, DC97]",4618,2,Staging
1,Aardewerk 1 backup,[DB34],4465,1,Staging
2,VONDSTENLIJST,"[DC112, DC97, DB34, DC93, DC11]",2121,5,Staging
3,Controle vondsten,[DB34],1466,1,Staging
4,DIAOPGRAVING,"[DC93, DB34, DC97]",882,3,Staging


In [5]:
df = df_files.merge(df_staging, on=['Table', 'Aantal_Records', 'Aantal_tabellen'], how='outer', suffixes=("_files", "_staging"))
df[df.Stage_files.isnull() | df.Stage_staging.isnull() ]

,Table,Projecten_files,Aantal_Records,Aantal_tabellen,Stage_files,Projecten_staging,Stage_staging
36,AARDEWERK 1&2 VOORWERP,[DB34],96,1,Files,NaN,NaN
55,FOTO,NaN,88,2,NaN,"[DC154, DC179]",Staging
